# E+T GILLESPIE'S ALGORITHM

##*Create directory to store results*
Before running the experiment, connect to Google Drive using the URL and following the instruction.

Change both the paths below to create a directory. Attention: the two paths has to be the same.

Notice: if you want to print the results only on the terminal there's no need to run this section.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')        # Connect your drive

In [ ]:
MY_DIR = 'drive/MyDrive'

!mkdir 'drive/MyDrive'    # Create output directory: we will store the results there  


## *Libraries*

These are useful libraries for running the program.

In [ ]:
# LIBRARIES
import numpy as np
#np.random.seed(1) # Remove comment to produce replicable results

## *Start the experiment*
We define functions useful for the experiment and the function that performs the experiments.

In the experiment for each step we generate one random pair ($\tau$, $\mu$) according to the joint probability density function $P(\tau, \mu)$ by employing Monte Carlo techniques. Using $\tau$ and $\mu$ we advance time by $\tau$ and update the {$X_i$} values of those species involved in the reaction. Then, we update $h_{\nu}c_{\nu}$ (and $a$).

In [ ]:
#FUNCTION FOR THE EXPERIMENT


#Combinatorial factor computation
def compute_h(mu_reaction, X_list):

  h_result = 1

  for i in range (0,len(X_list)):

    if (mu_reaction[i]== 0):   # =0 case: it's not a reactant
      h_result = h_result 
   
    else:               # >0 case: it's a reactant 
      numerator   = 1 
      denominator = 1   
      for j in range(mu_reaction[i]):  
        numerator   = numerator * (X_list[i] - j) 
        denominator = denominator * (j+1)
      
      h_result = h_result * (numerator/denominator)

  #print(f"Reaction : {mu_reaction}, h computed : {h_result}")  # Remove comment to print h 
  return h_result


#Tau extraction
def extract_tau(a): 

  r1 = np.random.rand()     # A random number uniformly sampled from [0,1] interval
  tau = (1/a) * np.log(1/r1)

  return tau


#Mu extraction
def extract_mu(a, ch_list):

    r2 = np.random.rand()  # A random number uniformly sampled from [0,1] interval
    for index_nu in range(len(ch_list)): 
      a_nu = np.sum(ch_list[:index_nu+1]) 

      if (a_nu >= r2*a) :
        mu_index = index_nu     

        return mu_index





#EXPERIMENT
def perform_experiment(ch, X, times, mu):

  ch_list = ch.copy()           # Initialization
  X_list = X.copy()
  sampling_times = times.copy()
  mu = mu.copy()
  a = np.sum(ch_list)   
  t = 0
  results_list = []
  stop_loop = False

  while (stop_loop == False):  

    while (t >= sampling_times[0]) :

      results_list.append(X_list)  
      #print(f"Printing informations at sampling time t : {round(sampling_times[0],5)}")    # Remove comment to print configurations
      #print(f"System configuration (X_list) : {X_list}")   
      sampling_times.pop(0) 
    
    tau = extract_tau(a)    # Tau generation
    
    mu_index = extract_mu(a, ch_list)     # Mu generation

    
    current_reaction = np.array(mu[mu_index][0:len(X)])-np.array(mu[mu_index][len(X):2*len(X)])   # The reaction occurred: update
    X_list = np.array(X_list) - np.array(current_reaction)  
    ch_list = []

    for reaction_index, reaction in enumerate(mu):

      ch_list.append(c_list[reaction_index] * (compute_h(reaction, X_list)))
    
    a = np.sum(ch_list)

    #print(f"\n  Reaction time : {t} \n")      # Remove comment to print reaction time
    t = t + tau                            
   
    if (t >= T_STOP or np.sum(ch_list) == 0):     # Stop loop condition
      results_list.append(X_list) 

      if len(results_list) < N_SAMPLES+2 :

        diff = N_SAMPLES+2 - len(results_list)

        for instant in range(diff):

          results_list.append(X_list)
          #print(f"Printing informations at sampling time t : {round(sampling_times[0],2)}")  # Remove comment to print configurations
          #print(f"System configuration (X_list) : {X_list}")
          sampling_times.pop(0) 

      stop_loop = True
  
  
  print(f"\n------------ EXPERIMENT COMPLETED -------------\n\n\n")
  return results_list   # Storing results of all experiments


## *Drop results into file*

This code generates a file where:
* For each time sample there is a new line
* Lines are organized as:  $t_{sample}, X_1, X_2, X_3, ..., X_N$

Notice: if you want to print the results only on the terminal there's no need to run this section.

In [ ]:
def drop_results_into_file(results_list, file_num):
  file_pos = MY_DIR + 'results_' + str(file_num) + '.txt'
    with open(file_pos, 'w') as f:
      for index, item in enumerate(results_list):
          newline = str(round(index*JUMP_LEN,2)) 
          for index_item in range(len(item)):
              newline = newline + '\t' + str(item[index_item])
          newline = newline + '\n'
          f.write(newline)

## *Launch the experiments!*
Here is where the inputs have to be inserted.

The program needs:

*   The number of simulations       (positive integer)
*   The number of samplings         (positive integer)
*   The sampling time interval      (positive)
*   Initial species configurations  (not negative integer entries)
*   The reaction parameters         (positive entries)
*   The stechiometric matrix        (not negative integer entries)

For the stechiometric matrix :

number of rows = lenght of reaction parameters vector

number of columns = 2 * lenght of initial number of species vector






In [ ]:

#INITIALIZATION 
N_EXPERIMENTS = 5000          # Number of simulations
N_SAMPLES   = 500            # Number of samplings
JUMP_LEN    = 0.08           # Sampling times interval


X = [1000, 0]              # Initial number of molecules for each chemical species


c_list = [0.2]          # Reaction parameters 
 
mu = [[1,0,0,1],
      ]                       # Stechiometric constants matrix 

T_STOP = N_SAMPLES * JUMP_LEN
times = [JUMP_LEN*elem for elem in range(N_SAMPLES+1)]
X = np.array(X)
ch = [] 
for reaction_index, reaction in enumerate(mu):
  ch.append(c_list[reaction_index] * (compute_h(reaction, X)))

# LAUNCH IT!
for i in range(1, N_EXPERIMENTS+1):
  results_list = perform_experiment(ch, X, times, mu)
  drop_results_into_file(results_list, i)  # Comment if file writing is unabled